In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from time import time

# Face Recognition

In this challenge, we will
* use PCA to **compress black & white images of famous people**  
* use our compressed images as samples for a classification task

This time, contrary to the previous K-means challenge:

Instead of performing unsupervised learning on **one image** to find patterns **between its pixels** to reduce its unique color numbers, we will work on a dataset of **multiple B&W images** to:

- find common patterns **between all images**
- reduce the number of "principal features" that describe them

More precisely, we will try to express each image of our dataset as a **linear combination of principal components** (principal images in this case, if you will) using PCA.

To compress our images, we will then **zero out the smallest principal components** and keep only the most important ones in the equation. Each "reduced linear combination" will represent an image that has been compressed.  

Luckily, because we only removed the least important components, our lower-dimensional projection of the dataset will preserve the maximal data variance between images, so we should still be able to recognize which person is in each image.

## 1) Load Data

❓ Run the cell below to download a copy of the famous LFW dataset provided by `Sklearn`.

In [ ]:
# Download data, should take around 2min
!curl https://wagon-public-datasets.s3-eu-west-1.amazonaws.com/05-Machine-Learning/06-Unsupervised-Learning/face_recognition/data.zip > data.zip

Once downloaded, the next cell will unzip the downloaded file into a `data` folder. It should look like this:

```bash
data
└── lfw_home
    ├── joblib
    ├── lfw
    ├── pairs.txt
    ├── pairsDevTest.txt
    └── pairsDevTrain.txt
```

In [ ]:
!unzip -q data.zip
!tree -L 2 data

In [ ]:
# from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(data_home='data', min_faces_per_person=70, resize=0.4, funneled=False)

💡 The **faces** object contains the following:
- `faces.images`: images as matrices of **50 x 37 pixels** you can plot 
- `faces.data`: flattened version of size **1850 x 1** *(50 x 37=1850)* 
- `faces.target`: number index representing a class among 7

❓ Run the cells below to check some basic facts about your data and see some images

In [ ]:
print(f"- Images shape: {faces.images.shape}")
print(f"- Data (flattened images) shape: {faces.data.shape}")
print(f"- Target shape: {faces.target.shape}")
print(f"- Number of classes: {np.unique(faces.target).shape}")
print(f"- Each class is a famous person: {', '.join(faces.target_names)}")

In [ ]:
# Let’s see some faces.
fig = plt.figure(figsize=(7,10))

for i in range(15):
    plt.subplot(5, 5, i + 1)
    plt.title(faces.target_names[faces.target[i]], size=11)
    plt.imshow(faces.images[i], cmap=plt.cm.gray)
    plt.xticks(())
    plt.yticks(())

plt.tight_layout()

**Disclaimer**: we are aware that this dataset is not diverse at all and we apologize in advance. However, it is very well suited to understand PCA (low pixel count, B&W, only a few categories, faces well centered in the pictures, etc.) so please keep moving forward in this challenge to understand the notion of "principal component"!

## 2) Compression with PCA

We have **1288** observations (images) and **1850** features (50 × 37 pixels).

Having so many features for so few observations is not great in Machine Learning; as a rule of thumb, you may want at least: $n_{features} << \sqrt{n_{observations}}$.

**PCA** can help reduce these features to a more manageable size while maintaining most of the information in the data.


❓ Fit a `PCA` on your **flattened images** to reduce their dimensions to 150 components  
👉 Store your fitted `PCA` in a variable named **pca**  
👉 Then assign their transformation to **data_projected**

In [ ]:
# YOUR CODE HERE

The images were projected onto the first 150 principal components only. 

Again, what we call components are **directions of maximum variance** of the data. 

Now, we don't need 1850 pixels anymore to describe each image, but only 150 values 🤓  

A gain by factor of $\frac{1850}{150} = 12$ 🚀  

❓ Look at the shape of your components, and make sure you understand what it represents  
❓ Look at the shape of your first component, again make sure you understand what it represents  

In [ ]:
# YOUR CODE HERE

Your first component is a vector of 1850 values.  
We now have 150 components of 1850 values each.

One face is described as a linear combination of those components.

Let's reconstruct one image from its reduced representation to see how it works.

❓ Use `inverse_transform` on your **data_projected** to reconstruct your compressed images  
👉 Store the result in **data_reconstructed**

In [ ]:
# YOUR CODE HERE

❓ Plot the 13th picture (George W. Bush) of the reconstructed dataset, and compare it with the original one. 

<details>
    <summary>💡Hint</summary>
Reshape the flattened data into an image of 50 x 37 pixels
</details>

In [ ]:
# YOUR CODE HERE

❓ Run the cell below to see a selection of reconstructed images

In [ ]:
fig = plt.figure(figsize=(7, 10))

for i in range(15):
    plt.subplot(5, 5, i + 1)

    # Display each image with a title, which we grab from the dataset
    plt.title(faces.target_names[faces.target[i]], size=11)
    plt.imshow(pca.inverse_transform(data_projected)[i].reshape((50,37)), cmap=plt.cm.gray)

    # Remove plot ticks
    plt.xticks(())
    plt.yticks(())
    
plt.tight_layout()

### 🧪 Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('projection', shape=data_projected.shape)
result.write()
print(result.check())

## 3) Investigate your Principal Components

❓ Plot an image that corresponds to the *\"mean\"* face of the whole dataset  
👉 Use a `gray` color map for your plots in this section

<details>
    <summary>💡Hint</summary>
    
You can use `pca.mean_` or `faces.data.mean(axis=0)`  
You will also need some reshaping to be able to plot it as an image
</details>

❓ Plot the images corresponding to the **first 5** principal components  

In [ ]:
# YOUR CODE HERE

☝️ Each PC is a flattened "image" of 1850 pixels  
We merely reshaped them to be able to visualize them as normal images

👇 Below is a list of definitions of these Principal Components  

❓ **Read them carefully and make sure you understand them**, otherwise consider raising a ticket 🎟️ 

💡 Your first PCs are the **most important _directions_** on your 1850-feature observations

💡 They are the most important **_linear combinations_** of your 1850 pixels

💡 The ones which **preserve the most _variance_** when your dataset of pictures is projected onto them  

💡 The first few PCs are the **regions of the 2D pixel grid that contain the _most variation_** between your 1288 images

❓ Plot the images corresponding to the **last 5** principal components  

In [ ]:
# YOUR CODE HERE

❓ Run the cell below to plot several images corresponding to principal components

In [ ]:
n_rows, n_cols = 3, 5
fig, axs = plt.subplots(n_rows, n_cols, figsize=(16, 9))

for i in range(n_rows * n_cols):
    ax = axs[i // n_cols, i % n_cols]
    ax.set_title(f'PC {i * 10 + 1}', size=12)
    ax.set_xticks(()), ax.set_yticks(())
    ax.imshow(pca.components_[i * 10].reshape(50, 37), cmap='gray')

plt.tight_layout()

☝️ Take some time to look at the PCs and strengthen your intuition on what they represent  

Notice that the first PCs capture the biggest/simplest patterns that explain most of the difference between images:
- Orientation of the face: looking left, right, up, or down
- Size of the face, mouth, nose, and eyes

While the last PCs capture the smallest/most detailed patterns:
- The shape of the mouth (moving or still)
- The structure of the chin

Every image can be represented by the "mean face" plus a linear combination of the 150 "PC faces".

If you want to go further check the optional section **Reconstruction of an Original Image**.

## 4) Choose the Optimal Number of Components

We encounter, as is often the case in Machine Learning, a trade-off ⚖️

**Lots of components** will give a compressed image that is:  
🙂 Close to the original image in terms of quality  
🙁 Not significantly lighter than the original image

**A few components** will give a compressed image that is:  
🙂 Significantly lighter than the original
🙁 Far from the original image in terms of quality

It is very important to find how many components are needed to describe the data without losing too much information  

We can determine this visually by plotting the cumulative sum of explained variance ratio as a function of the number of components  

This information is stored in the `explained_variance_ratio_` attribute of a fitted `PCA` object from `sklearn`   

❓ Plot the cumulative sum of explained variance ratio against the number of components

In [ ]:
# YOUR CODE HERE

☝️ This curve quantifies how much of the total variance is contained within the first components  

❓ Run the cell below and take some time to confirm the statements with your understanding of the graph:  
- The **first component** alone is enough to explain close to **20% of the variance**
- The first **25 components** are enough to explain close to **75% of the variance**
- We need about **94 components** to describe **90% of the variance**

In [ ]:
plt.figure(figsize=(16, 9))

# Plot the data
plt.plot(range(1, pca.n_components_+1), np.cumsum(pca.explained_variance_ratio_))

# Set labels
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

# Display grid in the background
plt.grid()

# Set the limits for the axes
plt.xlim((0, 151))
plt.ylim((0, 1))

# Add lines to the plot
plt.hlines(
    y=[.75, .9],
    xmin=[-5, -5],
    xmax=[25, 94],
    linestyles='dotted',
    colors=['red', 'orange'],
    linewidth=2
)
plt.vlines(
    x=[25, 94],
    ymin=[0, 0],
    ymax=[.75, .9],
    linestyles='dotted',
    colors=['red', 'orange'],
    linewidth=2
);

❓ What is the smallest number of components you need to keep to get _at least_ 80% of the variance?  
👉  Assign your answer to a variable named `minimal_pc_count`

In [ ]:
# YOUR CODE HERE

### 🧪 Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('components', min_pc = minimal_pc_count)
result.write()
print(result.check())

## 5) Classify Images (PCA as Feature Engineering)

It is time to use **PCA** as a tool for **supervised ML**  .

Here is your brief 👇

Given a picture of the face of a famous person among a selection, your model should be able to tell to whom the face belongs.  

Translating this brief into ML terms 👇
- Your samples are images
- Your features are their pixels
- Your target is a class among several (7)

❓ Cross-validate a model of your choice, suited for the classification task at hand    
👉 Record the time needed to train and evaluate your model

<details span='markdown'>
    <summary>💡Hint </summary>
You can use the following method to record execution time:
    
```python
from time import time
start = time()
# CODE for which you want to record execution time
execution_time = time() - start
```
</details>

In [ ]:
# YOUR CODE HERE

❓ Follow the same steps, this time using the projections of your images as features  

In [ ]:
# YOUR CODE HERE

👉 Compare your scores and execution times  

The quality of your model should have *slightly* decreased.  
The time needed to *\"choose\"* (train and evaluate) the model, however, should have *greatly* decreased!

From a business point of view, this is a great achievement 🏆  
As you will discover during the ML Ops module, training models comes at a cost 💸🙈

## 6) Search for the Optimal Number of Components

*This time, the Machine Learning way: Grid Search*.

💡 Now that we have a supervised (features-target) ML setting, we can grid search the optimal number of components.

❓ Before proceeding, hold out 30% of your data as a test set  

👉 As usual, assign your split data to `X_train`, `X_test`, `y_train`, `y_test`  
👉 In your `train_test_split`, use `random_state=42` to compare results with your buddy

💡 We will **select** our model by **cross-validating** on the **train set**  
Then we will **assess** our model by **scoring** it on the **test set**

In [ ]:
# YOUR CODE HERE

💡 A grid search calls for a pipeline  
❓ Use [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) to create a pipeline with two steps:
- A `PCA`, no need to choose the number of components now
- The [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) algorithm as an estimator

In [ ]:
# YOUR CODE HERE

❓ Create a cross-validated grid search that uses your pipeline  
👉 Search only for the number of components for your `PCA` among these options: `[50, 100, 200, 300]`  

In [ ]:
# YOUR CODE HERE

❓ Print the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report) of your best model  
👉 Use the `best estimator` from your grid search to obtain predictions from **X_test**  
👉 Use these predictions against **y_test** to print your classification report

In [ ]:
# YOUR CODE HERE

You may get an UndefinedMetricWarning if for one of the classes, there are no correct predictions.

❓ How many components give the best score?  
👉 Assign the value to a variable named **best_n_components**

In [ ]:
# YOUR CODE HERE

### 🧪 Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('search_components', best_pc=best_n_components)
result.write()
print(result.check())

## 7) What about Scaling, Balancing and Tuning?

*The complete Machine Learning pipeline*

We focused extensively on PCA but there are 3 ML methods you can use to enhance your score:
- Scale your data before applying PCA
- Use some form of balancing as your classes are not balanced
- Grid search for optimal hyperparameters for your estimator

Let's do it and see how using `PCA` alongside the ML tricks we have seen so far will help us achieve a higher score

❓ Run the cell below to see both your baseline and base score obtained with only PCA + SVC

In [ ]:
baseline = pd.Series(y).value_counts(normalize=True).max()

score_base = cross_validate(
    make_pipeline(
        PCA(n_components=best_n_components),
        SVC()
    ),
    X, y,
    scoring='accuracy',
    cv=3,
    n_jobs=-1
)['test_score'].mean()


print(f"""
    Accuracy scores:
    Baseline (frequency of most frequent class): {baseline: .2%}
    Base Model (PCA + SVC): {score_base:.2%}
""")

### Scaling

❓ Scale your data before reduction with a `PCA`  
👉 Build a pipeline that has 3 steps:
- Scaling with `StandardScaler`
- Reduction with `PCA` (use **best_n_components** from your earlier search)
- Prediction with `SVC` (keep all default arguments)

In [ ]:
# YOUR CODE HERE

❓ Cross-validate your pipeline on the full **X** and **y** over 3 folds  
👉 Store the mean score in **score_scaling**  
👉 Check your new score

In [ ]:
# YOUR CODE HERE

☝️ Up we go

### Balancing

❓ Check the spread of your target classes

In [ ]:
# YOUR CODE HERE

☝️ As you can see, your classes are highly unbalanced  

The most represented class appears on 41% of images  
While the least represented one appears on 5.5% of images  

This will cause your model to predict the most represented class too often, which will decrease
- the precision score for the most represented classes 
- the recall score for the least represented classes

❓ Train another pipeline that takes into account your class imbalance  
👉 Check the [documentation of the SVC estimator](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)  
👉 Find and use the parameter that helps with the class imbalance in your SVC  
👉 Store the mean score in **score_balanced** and check your new score

In [ ]:
# YOUR CODE HERE

🚀 Sky is the limit

### Fine-Tuning

❓ Fine-tune your model to find the combination of hyperparameters that yields the highest score  
👉 Search 3 hyperparameters maximum  
👉 For each one, search on 3 values maximum  
👉 Here is an indicative search dictionary you can use:
```python
grid = {
        'svc__kernel': ['rbf', 'poly', 'sigmoid'],
        'svc__gamma': [1e-4, 1e-3, 1e-2],
        'svc__C': [10, 1e2, 1e3]
}
```
ℹ️ These ranges of hyperparameters are examples and are not meant to offer the combination for the best model. Feel free to change the values!

In [ ]:
grid = {
    'svc__kernel': ['rbf', 'poly', 'sigmoid'],
    'svc__gamma': [1e-4, 1e-3, 1e-2],
    'svc__C': [10, 1e2, 1e3]
}

search = GridSearchCV(
    pipe_balanced, 
    grid,
    cv=3,
    scoring='accuracy'
)

search.fit(X, y)
score_tuned = search.best_score_
round(score_tuned, 5)

🚀 We increased accuracy by 10 points compared to our base model  

With a dimensionality reduction technique, such as `PCA`, it is faster to train, cross-validate and fine-tune our models  

Fine-tuning can be extremely long, being able to speed up the process by using a reduction on our data beforehand is a great advantage

### 🧪 Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'full_pipeline',
    score_scaled=score_scaling,
    score_balanced=score_balanced,
    score_tuned=score_tuned
)

result.write()
print(result.check())

🏁 **Don't forget to push your notebook.**  

Proceed with the challenges of the day and come back here if you have time 😉

<details>
  <summary markdown='span'>🍔 Food for thought</summary>

You can try to compare the result of the PCA-preprocessed classification with the one that was not preprocessed to see if
1. it is quicker
2. it is better
3. it helps find a linear separation
</details>

## 8) (Optional) Reconstruction of an Original Image

👉 Study the cells below which reconstruct the image step by step without `inverse_transform`  

👉 We start by selecting a single image for the example

In [ ]:
# Let's reconstruct the 13th image
image_original = faces.images[12];
image_compressed = data_projected[12];

plt.imshow(image_original, cmap='gray');

👉 We manually do the sum of multiplications $X\_reconstructed_{i} = \sum_{i=1}^{n\_components}{X_{projected_i} * W_i}$  
$W_i$ being the `i-th principal component`  

In [ ]:
# We start the reconstruction from the mean of all images
image_reconstructed = pca.mean_.copy(); 

# Then, reconstruct the image by computing the sum of every 150 entries of its compressed representation, weighted by the corresponding principal components
reconstruction = list()

for i in range(pca.n_components_):
    image_reconstructed += pca.components_[i] * image_compressed[i]
    reconstruction.append(image_reconstructed.copy())

👉 We plot the reconstructed image alongside the original

In [ ]:
# Plot the original and the compressed image.
fig, ax = plt.subplots(1, 2, figsize = (5, 5))

# Original
ax[0].imshow(image_original, cmap='gray')
ax[0].set_title('Original Image')

# Reconstructed
ax[1].imshow(image_reconstructed.reshape(faces.images[0].shape), cmap='gray')
ax[1].set_title('Compressed reconstructed Image')

for ax in fig.axes:
    ax.axis('off')

plt.tight_layout()

👉 We plot the image at different steps of reconstruction

In [ ]:
# Plot several images at different steps of reconstruction
n_rows, n_cols = 3, 5
fig, axs = plt.subplots(n_rows, n_cols, figsize=(16, 9))
plt.suptitle('Image reconstructed using only...')

for i in range(n_rows * n_cols):
    ax = axs[i // n_cols, i % n_cols]
    ax.set_title(f'... {i * 10 + 1} PC', size=12)
    ax.set_xticks(()), ax.set_yticks(())
    ax.imshow(reconstruction[i * 10].reshape(50, 37), cmap='gray')

plt.tight_layout()